In [1]:
import numpy as np
import pandas as pd
import csv
from scipy import stats
import os
import datetime
import statsmodels.api as sm
import requests
import sys, json
import time
import random
import json
import re

In [2]:
# DATA PREP. READ IN FILE

df_whisk_main_char = pd.read_csv('../Datasets/single_malt_main.csv', index_col = False)
df_peated_main_char = pd.read_csv('../Datasets/single_malt_peated_main.csv', index_col = False)
df_main_char = pd.concat([df_whisk_main_char, df_peated_main_char]).reset_index(drop = True)

df_whisk_profiles = pd.read_csv('../Datasets/single_malt_profiles.csv', index_col = False)
df_whisk_profiles['Brand'] = df_whisk_profiles['Brand'].str.upper()
df_peated_profiles = pd.read_csv('../Datasets/single_malt_peated_profiles.csv', index_col = False)
df_peated_profiles['Brand'] = df_peated_profiles['Brand'].str.upper()
df_profiles = pd.concat([df_whisk_profiles, df_peated_profiles]).reset_index(drop = True)

df_whisk_distillers = pd.read_csv('../Datasets/distilleries.csv', engine = 'python', index_col = False)
df_whisk_distillers['Distillery'] = df_whisk_distillers['Distillery'].str.upper()

In [3]:
# JOIN AND INITIAL CLEAN 

df_main_full = pd.merge(df_main_char, df_profiles, on = ["Whisk_Name", "URL"], how="left")

for idx in range(len(df_main_full)):
    raw_text = df_main_full['Whisk_Name'].loc[idx]
    raw_text = re.sub(u"[Á]", 'A', raw_text)
    raw_text = re.sub(u"[É]", 'E', raw_text)
    raw_text = re.sub(u"[Í]", 'I', raw_text)
    raw_text = re.sub(u"[Ó]", 'O', raw_text)
    raw_text = re.sub(u"[ÚÜ]", 'U', raw_text)
    df_main_full['Whisk_Name'].loc[idx] = raw_text

C:\Users\aclou\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [4]:
df_main_full.loc[(1, 1401),]

,Whisk_Name,Whisk_Type,Whisk_Loc,Communal_Rating,Distiller_Score,URL,Num_Revs,Brand,Cost,Age,ABV,Flavor,Profile
1,GLENGLASSAUGH 30 YEAR,Single Malt,"Highlands, Scotland",4.38,NaN,https://distiller.com/spirits/glenglassaugh-30...,9,GLENGLASSAUGH,spirit-cost cost-5,30 YEAR,42.0,NaN,NaN
1401,LAPHROAIG 10 YEAR,Peated Single Malt,"Islay, Scotland",3.98,96.0,https://distiller.com/spirits/laphroaig-10,8777,LAPHROAIG,spirit-cost cost-3,10 YEAR,43.0,PEATY & VANILLA,"{""smoky"":70,""peaty"":90,""spicy"":50,""herbal"":20,..."


In [5]:
# BREAK OUT PROFILES INTO COLUMNS

if np.isnan(df_main_char['Distiller_Score'][0]):
    ls_of_ls = [float("nan")]*14
    ls_of_ls = [ls_of_ls]
else: 
    ls_of_ls = list(json.loads(df_profiles['Profile'][0]).values())
    ls_of_ls = [ls_of_ls]

for idx in range(1, len(df_main_char)): 
    if np.isnan(df_main_char['Distiller_Score'][idx]):
        list_append = [float("nan")]*14
    else: 
        list_append = list(json.loads(df_profiles['Profile'][idx]).values())
    ls_of_ls.append(list_append)

profiles_full = pd.DataFrame(ls_of_ls, columns = list(json.loads(df_profiles['Profile'][0]).keys())) # we know the first one has the keys

In [6]:
df_main_full = pd.concat([df_main_full.reset_index(drop = True), profiles_full], axis = 1)
df_main_full.head(3)

,Whisk_Name,Whisk_Type,Whisk_Loc,Communal_Rating,Distiller_Score,URL,Num_Revs,Brand,Cost,Age,...,oily,full_bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral
0,TULLIBARDINE THE MURRAY MARSALA FINISH,Single Malt,"Highlands, Scotland",4.19,86.0,https://distiller.com/spirits/tullibardine-the...,8,TULLIBARDINE,spirit-cost cost-3,NAS,...,40.0,35.0,40.0,45.0,0.0,0.0,20.0,20.0,35.0,10.0
1,GLENGLASSAUGH 30 YEAR,Single Malt,"Highlands, Scotland",4.38,NaN,https://distiller.com/spirits/glenglassaugh-30...,9,GLENGLASSAUGH,spirit-cost cost-5,30 YEAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LEDAIG 13 AMONTILLADO CASK FINISH,Single Malt,"Islands, Scotland",3.84,NaN,https://distiller.com/spirits/ledaig-13-amonti...,9,NaN,spirit-cost cost-4,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# CLEAN UP AGE + COST

age_final = [re.sub(r'YEAR(.*)','', str(x)) for x in df_main_full['Age']]
age_final = [re.sub(r'NAS(.*)','NAS', str(y)) for y in age_final]
age_final = [re.sub(r'nan','NAS', str(y)) for y in age_final]
age_final = [str(y).strip() for y in age_final]
df_main_full['Age'] = age_final

cost_final = [re.sub(r'spirit-cost cost-','', str(x)) for x in df_main_full['Cost']]
cost_final = [re.sub(r'NAS(.*)','NAS', str(y)) for y in cost_final]
cost_final = [re.sub(r'nan','NAS', str(y)) for y in cost_final]
cost_final = [str(y).strip() for y in cost_final]
df_main_full['Cost'] = cost_final

In [8]:
# CLEAN UP DISTILLERS
distiller = []
for idx in range(len(df_main_full)):
    distiller_name = [distiller for distiller in df_whisk_distillers['Distillery'] if distiller in df_main_full['Whisk_Name'][idx]]
    if len(distiller_name) == 0:
        distiller.append("MISSING")
    else:
        distiller.append(distiller_name[0])

df_main_full['Distiller'] = distiller
df_main_full['Distiller'] = np.where(df_main_full['Distiller'] == "MISSING", \
                                     df_main_full['Brand'], df_main_full['Distiller'])

In [9]:
df_main_full[df_main_full['Distiller'].isnull()] # MISSING DISTILLERS

,Whisk_Name,Whisk_Type,Whisk_Loc,Communal_Rating,Distiller_Score,URL,Num_Revs,Brand,Cost,Age,...,full_bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral,Distiller
2,LEDAIG 13 AMONTILLADO CASK FINISH,Single Malt,"Islands, Scotland",3.84,NaN,https://distiller.com/spirits/ledaig-13-amonti...,9,NaN,4,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,SHIELDAIG SINGLE MALT SCOTCH WHISKY OLOROSO CA...,Single Malt,"Speyside, Scotland",3.25,NaN,https://distiller.com/spirits/shieldaig-single...,9,NaN,2,NAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,GLEN FOHDRY DARAICH IAPANACH,Single Malt,"Speyside, Scotland",3.43,NaN,https://distiller.com/spirits/glen-fohdry-dara...,7,NaN,3,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,SILVER SEAL 12 YEAR,Single Malt,"Speyside, Scotland",3.75,NaN,https://distiller.com/spirits/silver-seal-12-year,7,NaN,1,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,ISLAY GOLD ORLA,Single Malt,"Islay, Scotland",2.82,NaN,https://distiller.com/spirits/islay-gold-orla,7,NaN,2,NAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,CU BOCAN CREATION 1,Single Malt,"Highland, Scotland",3.43,NaN,https://distiller.com/spirits/cu-bocan-creation-1,7,NaN,3,NAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,KIRKWALL BAY,Single Malt,"Orkney Isles, Scotland",3.43,NaN,https://distiller.com/spirits/kirkwall-bay,8,NaN,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,GELFLING WING SEALANT (36.160 SMWS),Single Malt,"Speyside, Scotland",3.96,NaN,https://distiller.com/spirits/gelfling-wing-se...,6,NaN,4,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,LISMORE 18 YEAR,Single Malt,"Speyside, Scotland",3.42,NaN,https://distiller.com/spirits/lismore-18-year,6,NaN,2,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,THE HALF CENTURY BLEND BATCH 3,Single Malt,Scotland,4.54,NaN,https://distiller.com/spirits/the-half-century...,6,NaN,5,NAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# SET Missing Distilleries as OTHER for now? 
df_main_full['Distiller'] = df_main_full['Distiller'].fillna('OTHER')

In [11]:
# CLEAN UP Whiskey Location

df_main_full['Whisk_Loc'] = df_main_full['Whisk_Loc'].str.upper()
df_main_full['Whisk_Loc'].unique()

loc_final = [re.sub(r'[.,!?]','', str(x)) for x in df_main_full['Whisk_Loc']]
loc_final = [re.sub(r'SCOTLAND(.*)','', str(y)) for y in loc_final]
loc_final = [re.sub(r'nan','NAS', str(y)) for y in loc_final]
loc_final = [re.sub(r'I[\s]*S[\s]*L[\s]*A[\s]*Y', 'ISLAY', str(y)) for y in loc_final]
loc_final = [re.sub(r'SPEY[ESIDFR]*', 'SPEYSIDE', str(y)) for y in loc_final]
loc_final = [re.sub(r'HI[GHN]*LAN[DSFER]*', 'HIGHLAND', str(y)) for y in loc_final]
loc_final = [re.sub(r'CAM[BP]*EL[LTOWN]*', 'CAMPBELTOWN', str(y)) for y in loc_final]
loc_final = [str(y).strip() for y in loc_final]

df_main_full['Whisk_Loc'] = loc_final

In [14]:
# SAVE RESULTS 
df_main_full.to_csv('../Datasets/whiskeys_full_df_v2.csv', index = False)